In [2]:
import openpyxl
import gurobipy as gp
from gurobipy import GRB, quicksum
from collections import defaultdict

EXCEL_PATH = r"C:\TAOR\Tutor Allocation 2023-24 - Lars 1.xlsx"
SHEET_TA  = "Tutor Allocation 2023-24"
SHEET_TUT = "Tutors"
SHEET_COU = "Courses"

def safe_float(val):
    try:
        return float(val)
    except (ValueError, TypeError):
        return 0.0

def parse_time_session(s):
    s = s.strip()
    if not s:
        return None
    parts = s.split()
    if len(parts) < 2:
        return None
    day = parts[0]
    try:
        times = parts[1].split('-')
        start = int(times[0].split(':')[0])
        end = int(times[1].split(':')[0])
        return (day, start, end)
    except:
        return None

def has_time_conflict(times1, times2):
    for (d1, s1, e1) in times1:
        for (d2, s2, e2) in times2:
            if d1 == d2 and not (e1 <= s2 or e2 <= s1):
                return True
    return False

def read_sheet_TA(excel_path, sheet_name):
    wb = openpyxl.load_workbook(excel_path, data_only=True)
    sheet = wb[sheet_name]
    rows = list(sheet.iter_rows(values_only=True))
    wb.close()
    if not rows:
        return []
    header = rows[0]
    body = rows[1:]
    colTutor = header.index("Tutor")
    colRole = header.index("Role")
    colPosition = header.index("Position")
    colGroup = header.index("Group")
    colCourseCode = header.index("Course Code")
    colCourseName = header.index("Course Name")
    colSess1 = header.index("Session Day & Time (1)")
    colSess2 = header.index("Session Day & Time (2)")
    colSess3 = header.index("Session Day & Time (3)")
    colSess4 = header.index("Session Day & Time (4)")
    colSemester = header.index("Semester")
    colTutorNeed = header.index("# of Tutor workshops")
    colTtotal = header.index("Tutor Total Hrs")
    colGrandTotal = header.index("Grand Total Hrs")
    ta_rows = []
    for row in body:
        if all(x is None for x in row):
            continue
        d = {
            "Tutor": str(row[colTutor]) if row[colTutor] else "",
            "Role": str(row[colRole]) if row[colRole] else "",
            "Position": str(row[colPosition]) if row[colPosition] else "",
            "Group": str(row[colGroup]) if row[colGroup] else "",
            "CourseCode": str(row[colCourseCode]) if row[colCourseCode] else "",
            "CourseName": str(row[colCourseName]) if row[colCourseName] else "",
            "Session1": str(row[colSess1]) if row[colSess1] else "",
            "Session2": str(row[colSess2]) if row[colSess2] else "",
            "Session3": str(row[colSess3]) if row[colSess3] else "",
            "Session4": str(row[colSess4]) if row[colSess4] else "",
            "Semester": str(row[colSemester]) if row[colSemester] else "",
            "TutorNeed": safe_float(row[colTutorNeed]),
            "T_total_hrs": safe_float(row[colTtotal]),
            "Grand_total_hrs": safe_float(row[colGrandTotal])
        }
        ta_rows.append(d)
    return ta_rows

def read_sheet_Tutors(excel_path, sheet_name):
    wb = openpyxl.load_workbook(excel_path, data_only=True)
    sheet = wb[sheet_name]
    rows = list(sheet.iter_rows(values_only=True))
    wb.close()
    if not rows:
        return {}
    header = rows[0]
    body = rows[1:]
    colStaff = header.index("Staff Name")
    colS1Load = header.index("S1 Load")
    colS2Load = header.index("S2 Load")
    tutors_dict = {}
    for row in body:
        if all(x is None for x in row):
            continue
        name = str(row[colStaff]) if row[colStaff] else ""
        if not name:
            continue
        tutors_dict[name] = {
            "S1Load": safe_float(row[colS1Load]),
            "S2Load": safe_float(row[colS2Load])
        }
    return tutors_dict

def read_sheet_Courses(excel_path, sheet_name):
    wb = openpyxl.load_workbook(excel_path, data_only=True)
    sheet = wb[sheet_name]
    rows = list(sheet.iter_rows(values_only=True))
    wb.close()
    if not rows:
        return {}
    header = rows[0]
    body = rows[1:]
    colCode = header.index("Code")
    colNoTreq = header.index("No T Required")
    courses_dict = {}
    for row in body:
        if all(x is None for x in row):
            continue
        code = str(row[colCode]) if row[colCode] else ""
        if not code:
            continue
        noTreq = safe_float(row[colNoTreq])
        courses_dict[code] = {"NoTreq": noTreq}
    return courses_dict

def read_sheet_Lookups(excel_path, sheet_name):
    wb = openpyxl.load_workbook(excel_path, data_only=True)
    sheet = wb[sheet_name]
    rows = list(sheet.iter_rows(values_only=True))
    wb.close()
    return [list(r) for r in rows if any(x is not None for x in r)]

def unify_workshops(ta_rows):
    groups = defaultdict(list)
    for row in ta_rows:
        key = (row["CourseCode"], row["Semester"], row["Session1"].strip(), row["Session2"].strip(), row["Session3"].strip(), row["Session4"].strip())
        groups[key].append(row)
    workshop_list = []
    workshop_need = []
    for key, lines in groups.items():
        total_need = sum(r["TutorNeed"] for r in lines)
        if total_need < 1:
            total_need = 1
        workshop_list.append(key)
        workshop_need.append(total_need)
    return workshop_list, workshop_need

def build_ilp_model(ta_rows, tutors_dict, courses_dict):
    model = gp.Model("TutorAllocationFull")
    workshop_list, workshop_need = unify_workshops(ta_rows)
    W = len(workshop_list)
    tutor_set = list(tutors_dict.keys())
    x = {}
    for t in tutor_set:
        for w in range(W):
            x[t, w] = model.addVar(vtype=GRB.BINARY, name=f"x_{t}_{w}")
    obj_expr = gp.LinExpr()
    for w, key in enumerate(workshop_list):
        grand_hrs = 0.0
        for row in ta_rows:
            kk = (row["CourseCode"], row["Semester"], row["Session1"].strip(), row["Session2"].strip(), row["Session3"].strip(), row["Session4"].strip())
            if kk == key:
                grand_hrs = row["Grand_total_hrs"]
                break
        for t in tutor_set:
            obj_expr.addTerms(grand_hrs, x[t, w])
    model.setObjective(obj_expr, GRB.MAXIMIZE)
    for w in range(W):
        model.addConstr(quicksum(x[t, w] for t in tutor_set) == workshop_need[w], name=f"WorkshopReq_{w}")
    for t in tutor_set:
        cap = tutors_dict[t]["S1Load"] + tutors_dict[t]["S2Load"]
        model.addConstr(quicksum(x[t, w] for w in range(W)) <= cap / 10.0, name=f"Workload_{t}")
    for t in tutor_set:
        model.addConstr(quicksum(x[t, w] for w in range(W)) <= 10, name=f"MaxWorkshops_{t}")
    course_to_w = defaultdict(list)
    for w, key in enumerate(workshop_list):
        course_code = key[0]
        course_to_w[course_code].append(w)
    for t in tutor_set:
        for c, wlist in course_to_w.items():
            model.addConstr(quicksum(x[t, w] for w in wlist) <= 1, name=f"UniqueCourse_{t}_{c}")
    workshop_times = []
    for w, key in enumerate(workshop_list):
        times = []
        for sess in key[2:]:
            p = parse_time_session(sess)
            if p:
                times.append(p)
        workshop_times.append(times)
    for t in tutor_set:
        for w1 in range(W):
            for w2 in range(w1 + 1, W):
                if has_time_conflict(workshop_times[w1], workshop_times[w2]):
                    model.addConstr(x[t, w1] + x[t, w2] <= 1, name=f"TimeConflict_{t}_{w1}_{w2}")
    model.optimize()
    if model.SolCount > 0:
        print("\n=== Optimal solution found ===")
        for t in tutor_set:
            assigned = []
            for w in range(W):
                if x[t, w].X > 0.5:
                    key = workshop_list[w]
                    ccode, sem, s1, s2, s3, s4 = key
                    cName = ""
                    for row in ta_rows:
                        kk = (row["CourseCode"], row["Semester"], row["Session1"].strip(), row["Session2"].strip(), row["Session3"].strip(), row["Session4"].strip())
                        if kk == key:
                            cName = row["CourseName"]
                            break
                    assigned.append(f"{ccode}({cName})")
            if assigned:
                print(f"Tutor {t} assigned to: {assigned}")
    else:
        print("No feasible solution or optimization was stopped.")
    return model, x

if __name__ == "__main__":
    ta_rows = read_sheet_TA(EXCEL_PATH, SHEET_TA)
    tutors_dict = read_sheet_Tutors(EXCEL_PATH, SHEET_TUT)
    courses_dict = read_sheet_Courses(EXCEL_PATH, SHEET_COU)
    model, x = build_ilp_model(ta_rows, tutors_dict, courses_dict)


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10870H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 59212 rows, 122122 columns and 488488 nonzeros
Model fingerprint: 0xde00d753
Variable types: 0 continuous, 122122 integer (122122 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 45295.000000
Presolve removed 48234 rows and 37180 columns
Presolve time: 0.53s
Presolved: 10978 rows, 84942 columns, 224829 nonzeros
Variable types: 0 continuous, 84942 integer (84942 binary)

Explored 0 nodes (0 simplex iterations) in 0.62 seconds (0.65 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 45295 

Optimal solution found (tolerance 1.00e-04)
Best obj